In [1]:
import pandas as pd
import numpy as np
import glob
from pyts.image import RecurrencePlot, GramianAngularField, MarkovTransitionField
from sklearn.metrics import accuracy_score
import gc
from keras.backend.tensorflow_backend import set_session
import os
import tensorflow as tf
from resnet_augmented import *

Using TensorFlow backend.
C:\Users\cyril\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\cyril\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\cyril\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\cyril\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()
_SESSION = None
if _SESSION is None:
    if not os.environ.get('OMP_NUM_THREADS'):
        config = tf.ConfigProto(allow_soft_placement=True)
        config.gpu_options.allow_growth=True
    else:
        num_thread = int(os.environ.get('OMP_NUM_THREADS'))
        config = tf.ConfigProto(intra_op_parallelism_threads=num_thread,
                                allow_soft_placement=True)
        config.gpu_options.allow_growth=True
    _SESSION = tf.Session(config=config)
session = _SESSION
set_session(session)

In [3]:
files = glob.glob('data.*')
files_train = files[:-2]
files_test = files[1:-1]

In [4]:
columns = ['time', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9',
       'b10', 'b11', 'b12', 'b13', 'b14', 'b15', 'b16', 'b17', 'b18', 'b19',
       'b20', 'b21', 'b22', 'b23', 'b24', 'b25', 'b26', 'b27', 'b28', 'b29',
       'b30', 'b31', 'b32', 'b33', 'b34', 'b35', 'b36', 'b37', 'b38', 'b39',
       'b40', 'b41', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9',
       's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19',
       's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27', 's28', 's29',
       's30', 's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's39',
       's40', 's41', 's42', 's43', 's44', 's45', 's46', 's47', 's48', 's49',
       's50', 'y_1m', 'y_5m', 'y_10m', 'y_15m', 'y_30m', 'y_hat1', 'y_hat2',
       'y_hat3']
#for i in range(len(files_train)):
i=0
#File loading
if i == 0:
    df = pd.read_csv(files_train[i])
    df.columns = columns
else:
    df = pd.read_csv(files_train[i],names=columns)
test = pd.read_csv(files_test[i],header=None,names=columns)

#Preprocessing
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df[df.columns[0]])))#[-1000:]
target = df.y_10m
df = df.drop(['s19','s24','time',"y_1m","y_5m","y_10m","y_15m","y_30m","y_hat1","y_hat2","y_hat3"],axis=1)
df = df.fillna(0)

test = test.set_index(pd.DatetimeIndex(pd.to_datetime(test[test.columns[0]]))) #[:15000]
target_test = test.y_10m
test = test.drop(['s19','s24','time',"y_1m","y_5m","y_10m","y_15m","y_30m","y_hat1","y_hat2","y_hat3"],axis=1)
test = test.fillna(0)
test2 = np.array(test)

# transformer = RecurrencePlot()
# df_temp1 = transformer.fit_transform(df)
# transformer2 = GramianAngularField()
# df_temp2 = transformer2.fit_transform(df)
# transformer3 = MarkovTransitionField()
# df_temp3 = transformer3.fit_transform(df)
# df2 = (df_temp1+df_temp2+df_temp3)/3

X_train = np.array(df).reshape(df.shape + (1,1,))
y_train = (np.array(target).reshape(len(target),1) - np.mean(target))/np.std(target)

# df_temp1 = transformer.transform(test2)
# transformer2 = GramianAngularField()
# df_temp2 = transformer2.transform(test2)
# transformer3 = MarkovTransitionField()
# df_temp3 = transformer3.transform(test2)
# test2 = (df_temp1+df_temp2+df_temp3)/3

X_test = np.array(test2).reshape(test2.shape + (1,1,))
y_test = (np.array(target_test).reshape(len(target_test),1) - np.mean(target))/np.std(target)

x , y = build_resnet(X_train.shape[1:], 64, 1)
model = keras.models.Model(inputs=x, outputs=y)
print(model.summary())
optimizer = keras.optimizers.Adam()


if i == 0:
    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mse'])
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,
                  patience=2, min_lr=0.0001) 
    hist = model.fit(X_train, y_train, batch_size=512,epochs=20,
        #sample_weight=target_train**2+1e-16,
        #validation_data=(X_test, y_test),  #,target**2+1e-16
        verbose=1, callbacks = [reduce_lr])


build conv_x
build conv_y
build conv_z
Merging skip connection
build conv_x
build conv_y
build conv_z
Merging skip connection
build conv_x
build conv_y
build conv_z
Merging skip connection
        -- model was built.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 89, 1, 1)     0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 89, 1, 1)     4           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 89, 1, 64)    4160        batch_normalization[0][0]        
__________________________________________________________________________________________

Epoch 1/20
482380/482380 [==============================] - 3520s 7ms/step - loss: 0.9212 - mean_squared_error: 0.9212
Epoch 2/20
482380/482380 [==============================] - 3411s 7ms/step - loss: 0.6771 - mean_squared_error: 0.6771
Epoch 3/20
482380/482380 [==============================] - 3414s 7ms/step - loss: 0.4021 - mean_squared_error: 0.4021
Epoch 4/20
482380/482380 [==============================] - 3410s 7ms/step - loss: 0.2560 - mean_squared_error: 0.2560
Epoch 5/20
482380/482380 [==============================] - 3408s 7ms/step - loss: 0.1869 - mean_squared_error: 0.1869
Epoch 6/20
482380/482380 [==============================] - 3409s 7ms/step - loss: 0.1496 - mean_squared_error: 0.1496
Epoch 7/20
482380/482380 [==============================] - 3411s 7ms/step - loss: 0.1286 - mean_squared_error: 0.1286
Epoch 8/20
482380/482380 [==============================] - 3408s 7ms/step - loss: 0.1127 - mean_squared_error: 0.1127
Epoch 9/20
482380/482380 [======================

KeyboardInterrupt: 

In [6]:
model.save('resnet.h5')

In [5]:
y_pred = model.predict(X_test,verbose=1)

#print("accuracy_score" + str(accuracy_score(np.sign(np.array(target_test)),np.sign(y_pred))))
lin = np.linspace(0,1,5)
print(pd.DataFrame(y_pred).quantile(lin))
print("cor: " + str(np.corrcoef(target_test,y_pred)[0,1]))
pd.DataFrame({'index':test.index,'y_pred':y_pred,'target_test':target_test}).to_csv("resnet" + str(i) + ".csv")

482381/482381 [==============================] - 1286s 3ms/step
             0
0.00 -3.998086
0.25 -0.550768
0.50 -0.259584
0.75  0.031926
1.00  4.033814
cor: 0.04775162973248426


In [9]:
y_pred2 = (np.concatenate(y_pred)*np.std(target))+np.mean(target)
lin = np.linspace(0,1,5)
print(pd.DataFrame(y_pred2).quantile(lin))
print("cor: " + str(np.corrcoef(target_test,y_pred2)[0,1]))
pd.DataFrame({'index':test.index,'y_pred':y_pred2,'target_test':target_test}).to_csv("resnet" + str(i) + ".csv")

             0
0.00 -0.011433
0.25 -0.001620
0.50 -0.000791
0.75  0.000038
1.00  0.011430
cor: 0.0477516298340451


In [11]:
for i in range(1,len(files_train)):
    test = pd.read_csv(files_test[i],header=None,names=columns)
    test = test.set_index(pd.DatetimeIndex(pd.to_datetime(test[test.columns[0]]))) #[:15000]
    target_test = test.y_10m
    test = test.drop(['s19','s24','time',"y_1m","y_5m","y_10m","y_15m","y_30m","y_hat1","y_hat2","y_hat3"],axis=1)
    test = test.fillna(0)
    test2 = np.array(test)
    X_test = np.array(test2).reshape(test2.shape + (1,1,))
    y_test = (np.array(target_test).reshape(len(target_test),1) - np.mean(target))/np.std(target)
    y_pred = model.predict(X_test,verbose=1)
    y_pred = (np.concatenate(y_pred)*np.std(target))+np.mean(target)
    #print("accuracy_score" + str(accuracy_score(np.sign(np.array(target_test)),np.sign(y_pred))))
    lin = np.linspace(0,1,5)
    print(pd.DataFrame(y_pred).quantile(lin))
    print("cor: " + str(np.corrcoef(target_test,y_pred)[0,1]))
    pd.DataFrame({'index':test.index,'y_pred':y_pred,'target_test':target_test}).to_csv("resnet" + str(i) + ".csv")

482381/482381 [==============================] - 1282s 3ms/step
             0
0.00 -0.009758
0.25 -0.001316
0.50 -0.000424
0.75  0.000525
1.00  0.011951
cor: 0.06068300708464704
482381/482381 [==============================] - 1295s 3ms/step
             0
0.00 -0.007373
0.25 -0.000642
0.50  0.000394
0.75  0.001552
1.00  0.012609
cor: 0.016029771200674406
482381/482381 [==============================] - 1288s 3ms/step
             0
0.00 -0.007315
0.25 -0.000826
0.50  0.000159
0.75  0.001249
1.00  0.014424
cor: 0.016464439773049067
482381/482381 [==============================] - 1312s 3ms/step
             0
0.00 -0.008494
0.25 -0.001200
0.50 -0.000024
0.75  0.001264
1.00  0.012401
cor: 0.04104880350831984
482381/482381 [==============================] - 1304s 3ms/step
             0
0.00 -0.007429
0.25 -0.001944
0.50 -0.001255
0.75 -0.000576
1.00  0.005876
cor: 0.02038513644457559
482381/482381 [==============================] - 1298s 3ms/step
             0
0.00 -0.007302
0.25 -0.0

In [12]:
files = glob.glob('resnet*.csv')
for i,file in enumerate(files):
    df = pd.read_csv(file)
    df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df[df.columns[0]]))) 
    df =df.y_pred
    if i == 0:
        all_df = df
    else:
        all_df = pd.concat((all_df,df))
all_df.to_csv('all_resnet.csv')

C:\Users\cyril\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  # Remove the CWD from sys.path while we load stuff.


In [5]:
files = ['all_rocket.csv','all_attlstm.csv','all_resnet.csv']
df = pd.read_csv(files[0])
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df[df.columns[0]]))) 
df2 = pd.read_csv(files[1])
df2 = df2.set_index(pd.DatetimeIndex(pd.to_datetime(df2[df2.columns[0]]))) 
df3 = pd.read_csv(files[2])
df3 = df3.set_index(pd.DatetimeIndex(pd.to_datetime(df3[df3.columns[0]])))
all_df = (df.IF + df2.IF + df3.IF) /3
all_df.to_csv('all_att_rocket_resnet.csv')

In [3]:
files = 'IF.csv'
df4 = pd.read_csv(files)
df4 = df4.set_index(pd.DatetimeIndex(pd.to_datetime(df4[df4.columns[0]])))
df4

,Unnamed: 0,IF
Unnamed: 0,,
2012-08-24 09:35:03,2012-08-24 09:35:03,-0.000136
2012-08-24 09:35:06,2012-08-24 09:35:06,-0.000126
2012-08-24 09:35:09,2012-08-24 09:35:09,-0.000116
2012-08-24 09:35:12,2012-08-24 09:35:12,-0.000092
2012-08-24 09:35:15,2012-08-24 09:35:15,-0.000099
...,...,...
2019-10-23 14:29:48,2019-10-23 14:29:48,-0.000203
2019-10-23 14:29:51,2019-10-23 14:29:51,-0.000171
2019-10-23 14:29:54,2019-10-23 14:29:54,-0.000185


In [6]:
all_df = (df.IF + df2.IF + df3.IF + df4.IF) /4
all_df.to_csv('all.csv')

C:\Users\cyril\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
